# Task 4
This serves as a template which will guide you through the implementation of this task. It is advised to first read the whole template and get a sense of the overall structure of the code before trying to fill in any of the TODO gaps.
This is the jupyter notebook version of the template. For the python file version, please refer to the file `template_solution.py`.

First, we import necessary libraries:

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, DistilBertModel
from transformers import pipeline
from torch.optim import AdamW
#pipe = pipeline("text-classification")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Add any other imports you need here

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Depending on your approach, you might need to adapt the structure of this template or parts not marked by TODOs.
It is not necessary to completely follow this template. Feel free to add more code and delete any parts that are not required.

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
#model_transformer=model_transformer.to(DEVICE)

BATCH_SIZE = 16  # TODO: Set the batch size according to both training performance and available memory
NUM_EPOCHS = 4  # TODO: Set the number of epochs
MAX_LEN = 100

train_val = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/task4_hr35z9/train.csv")
test_val = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/task4_hr35z9/test_no_score.csv")

#len(test_val)
#test_val["title"]+". "+test_val["sentence"]
#test_val["title"][0]
#index=1
#cat=train_val.iloc[index]["title"]+". "+train_val.iloc[index]["sentence"]
#inputs = tokenizer(cat, return_tensors="pt")
#outputembedding = model_transformer(**inputs)
#outputembedding.last_hidden_state.squeeze().size()
#print(outputembedding)

cuda:0


In [5]:
# TODO: Fill out ReviewDataset
class ReviewDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe["title"]+" "+dataframe["sentence"]

        self.max_len = max_len

        self.targets = dataframe["score"]


    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = self.text[index]
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=False,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

        #return {
        #    'ids': torch.tensor(ids, dtype=torch.long),
        #    'mask': torch.tensor(mask, dtype=torch.long),
        #    'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long)}

In [6]:
class ReviewDataset1(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe["title"]+" "+dataframe["sentence"]

        self.max_len = max_len

        #self.targets = dataframe["score"]


    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=False,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long)
        }

        #return {
        #    'ids': torch.tensor(ids, dtype=torch.long),
        #    'mask': torch.tensor(mask, dtype=torch.long),
        #    'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long)}

In [7]:
with torch.no_grad():
  train_dataset = ReviewDataset(train_val,tokenizer,MAX_LEN)
  test_dataset = ReviewDataset1(test_val,tokenizer,MAX_LEN)


  train_loader = DataLoader(dataset=train_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=True, num_workers=2, pin_memory=True)
  test_loader = DataLoader(dataset=test_dataset,
                         batch_size=BATCH_SIZE,
                         shuffle=False, num_workers=2, pin_memory=True)
# Additional code if needed


In [10]:
# TODO: Fill out MyModule
class MyModule(nn.Module):
    def __init__(self):
        super(MyModule, self).__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.4)
        self.classifier = torch.nn.Linear(768, 1)

    def forward(self, input_ids, attention_mask):
    #def forward(self,preprocess):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)

        hidden_state = output_1[0]
        #print(hidden_state.size())
        pooler = hidden_state[:,0]
        #print(pooler.size())
        pooler = self.classifier(pooler)
        #pooler = F.relu(pooler)
        #pooler = self.dropout(pooler)
        #output = self.classifier(pooler)
        output=10*F.sigmoid(pooler)
        return output


model = MyModule().to(device)

In [11]:
# TODO: Setup loss function, optimiser, and scheduler
optimizer = AdamW(model.parameters(), lr=1e-4)
criterion = nn.MSELoss()
#optim = torch.optim.SGD(model.parameters(), lr=0.001,momentum=0.9)
scheduler = None


for epoch in range(NUM_EPOCHS):
    print(f'Current epoche:{epoch}')
    run_loss = 0
    model.train()

    for i,data in tqdm(enumerate(train_loader,0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        #token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        #model_frozen=DistilBertModel.from_pretrained("distilbert-base-uncased")
        #preprocess = model_frozen(input_ids=ids, attention_mask=mask)

        outputs = model(ids, mask).squeeze()
        #loss = outputs.loss
        optimizer.zero_grad()
        loss=criterion(outputs,targets)
        loss.backward()
        optimizer.step()
        #scheduler.step()
        run_loss = run_loss + loss.item()
        if i % 200 == 199:    # print every 200 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {run_loss / 200:.3f}')
            run_loss = 0

    torch.save(obj=model.state_dict(), f="/content/drive/MyDrive/Colab Notebooks/task4_hr35z9/model.pth")
        # TODO: Set up training loop

Current epoche:0


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2674: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length wi

[1,   200] loss: 5.018


401it [00:58,  6.70it/s]

[1,   400] loss: 3.204


...

401it [00:58,  6.69it/s]

[4,   400] loss: 0.488


601it [01:28,  6.82it/s]

[4,   600] loss: 0.519


782it [01:54,  6.81it/s]


In [12]:
model.eval()

results=[]
with torch.no_grad():
    for _, data in tqdm(enumerate(test_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        #print(ids)
        mask = data['mask'].to(device, dtype = torch.long)

        #print(mask)
        #token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)

        #targets = data['targets'].to(device, dtype = torch.float)
        outputs = model(ids, mask)
        #outputs=data
        #print(f"IDs: {ids}, Outputs: {outputs}")

        #fin_targets.extend(targets.cpu().detach().numpy().tolist())
        results.extend(outputs.cpu().detach().numpy().tolist())
    with open("/content/drive/MyDrive/Colab Notebooks/task4_hr35z9/result.txt", "w") as f:
        for val in results:
            f.write(f"{val[0]}\n")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2674: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific v

IDs: tensor([[ 1006, 12008,  1007,  ...,  2009,  2001,  2145],
        [ 2028,  1997,  2643,  ...,  2003,  4129,  2149],
        [ 2052,  5223,  2000,  ...,  1010,  1998,  1996],
        ...,
        [ 5236,  1024,  3954,  ...,  2005,  4450,  1010],
        [ 2023,  2208, 19237,  ..., 22662,  2041,  2006],
        [ 2145,  3403,  2005,  ...,  2094,  2544,  4107]], device='cuda:0'), Outputs: tensor([[1.8022],
        [9.5416],
        [0.5223],
        [2.9021],
        [4.8361],
        [5.2709],
        [5.7764],
        [5.8677],
        [4.1062],
        [7.5381],
        [9.4146],
        [4.0914],
        [9.5117],
        [0.2767],
        [3.0988],
        [5.0438]], device='cuda:0')
IDs: tensor([[ 2069,  2005,  1996,  ...,  2003,  5496,  1010],
        [24970,  4765,  4431,  ...,     0,     0,     0],
        [ 4276,  1996, 10961,  ...,     0,     0,     0],
        ...,
        [ 2005,  7987, 20211,  ...,  2944,  5194,  1012],
        [ 2204,  2466,  1010,  ...,  6257,  2009, 

7it [00:00, 12.88it/s]

IDs: tensor([[ 1037,  2047,  3921,  ...,  2065,  2017,  2215],
        [ 2190,  2189,   999,  ...,     0,     0,     0],
        [ 1996,  2774,  3768,  ...,  1010,  1998,  2014],
        ...,
        [ 2307,  2338, 24873,  ...,     0,     0,     0],
        [ 5745, 17134,  1045,  ...,     0,     0,     0],
        [ 1045,  2293,  2023,  ...,     0,     0,     0]], device='cuda:0'), Outputs: tensor([[4.9658],
        [9.6388],
        [2.5052],
        [9.4792],
        [3.0681],
        [8.9855],
        [1.3749],
        [9.2718],
        [3.7194],
        [4.3210],
        [5.9143],
        [3.3223],
        [2.6144],
        [8.1214],
        [5.1349],
        [7.9911]], device='cuda:0')
IDs: tensor([[ 3714,  2785,  2571,  ...,  6453,  2071,  2022],
        [ 1037,  6270,  2466,  ...,  1012,  2338,  5324],
        [14984,  1997, 24903,  ...,  1055,  2134,  1005],
        ...,
        [ 2052,  2022,  3819,  ...,  1005,  1055,  2525],
        [ 8875,  2038,  2904,  ...,  2000,  1996, 

13it [00:00, 18.63it/s]

...

IDs: tensor([[ 2028,  1997,  5297,  ...,     0,     0,     0],
        [ 2106,  2025,  2147,  ...,     0,     0,     0],
        [ 2307,  2793,  1045,  ...,  8694,  3813,  2791],
        ...,
        [ 2204,  1010,  2748,  ...,  2734,  2000,  2022],
        [ 2131,  2115,  2303,  ...,  3325, 11007,  1997],
        [ 2190,  3104,  2412,  ...,  1045,  2387,  1996]], device='cuda:0'), Outputs: tensor([[0.3491],
        [2.0883],
        [9.3571],
        [2.3843],
        [8.3387],
        [2.4118],
        [0.3537],
        [3.7495],
        [9.6292],
        [0.8708],
        [2.8521],
        [9.4518],
        [3.5220],
        [6.4714],
        [9.2642],
        [6.9419]], device='cuda:0')
IDs: tensor([[16795,  2594, 11086,  ...,     0,     0,     0],
        [ 2995,  2135,  1037,  ...,  3060,  4841,  1007],
        [ 4986,  2034,  2051,  ...,  6081,  2338,  7926],
        ...,
        [ 2067,  2000,  1996,  ...,  1012,  4231,  2668],
        [ 2987,  1005,  1056,  ...,  4937, 12237, 

58it [00:02, 22.24it/s]

IDs: tensor([[ 2763,  1037,  3293,  ...,  3835,  7680,  2005],
        [10346,  6799,  2100,  ...,     0,     0,     0],
        [10654,  5302,  3600,  ...,     0,     0,     0],
        ...,
        [ 8038,  1005,  2222,  ...,  5448,  1010,  2746],
        [22889, 10376,  3334,  ..., 24665, 12083,  5910],
        [ 8321, 15921,  1997,  ...,  3436,  2003,  2074]], device='cuda:0'), Outputs: tensor([[6.6578],
        [4.3824],
        [3.4748],
        [7.2658],
        [5.4961],
        [4.2444],
        [1.0843],
        [7.7235],
        [9.1947],
        [3.9704],
        [2.8895],
        [9.6558],
        [1.6057],
        [4.9114],
        [1.2036],
        [6.2303]], device='cuda:0')
IDs: tensor([[ 2123,  1005,  1056,  ...,     0,     0,     0],
        [ 2204,  3191,  1037,  ...,     0,     0,     0],
        [20810,  2004,  2035,  ...,  2593,  2288,  3737],
        ...,
        [ 2081,  1999,  2605,  ...,     0,     0,     0],
        [ 6881,  2143,  1010,  ..., 15265,  7716, 

63it [00:03, 19.61it/s]

IDs: tensor([[ 2513,  3807,  1011,  ...,  2020,  2025, 13115],
        [ 3492,  4658,  3242,  ...,  2055,  2321,  2781],
        [ 2025,  2012,  2035,  ...,  6820,  6593,  7028],
        ...,
        [ 4241,  7382,  3111,  ...,     0,     0,     0],
        [ 2200,  8796,  1004,  ...,     0,     0,     0],
        [ 5875,  1010,  2021,  ...,  3752,  2320,  1010]], device='cuda:0'), Outputs: tensor([[2.2353],
        [7.7742],
        [3.1908],
        [2.7084],
        [3.7248],
        [3.0229],
        [4.1876],
        [7.9428],
        [3.7029],
        [9.3764],
        [0.5765],
        [9.4716],
        [9.6524],
        [4.6965],
        [1.0172],
        [4.2933]], device='cuda:0')
IDs: tensor([[ 6581,  4031,  6581,  ...,  3899,  3046,  2028],
        [ 2047,  5440,  2201,  ...,  1012,  9826,  2295],
        [ 2009,  2573,  2021,  ...,  1996,  3754,  2000],
        ...,
        [ 6902,  2317, 14260,  ...,     0,     0,     0],
        [11771,  2201,  2007,  ...,  1996,  2316, 